In [1]:
import os
import pprint

from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
import time
import numpy as np
from dotenv import load_dotenv
from langchain_community.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceBgeEmbeddings


load_dotenv()


USER_AGENT environment variable not set, consider setting it to identify your requests.


False

In [ ]:
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    #model_name="jaimevera1107/all-MiniLM-L6-v2-similarity-es",
    model_name="google-t5/t5-small",
    model_kwargs={'device':'cpu', 'trust_remote_code': True}, 
    encode_kwargs={'normalize_embeddings': False, 'attn_implementation': "eager"},
)

Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-base-es and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.10.intermediate.dense.bias', 'encoder.layer.10.intermediate.dense.weight', 'encoder.layer.10.output.LayerNorm.bias', 'encoder.layer.10.output.LayerNorm.weight', 'encoder.layer.10.output.dense.bias', 'encoder.layer.10.output.dense.weight', 'encoder.layer.11.intermediate.dense.bias', 'encoder.layer.11.intermedi

In [3]:
import chromadb
from chromadb.utils import embedding_functions

# Use similarity searching algorithm and return 3 most relevant documents.
client = chromadb.PersistentClient('./db_subvenciones')

db = Chroma(client=client, collection_name="ayudas",embedding_function=huggingface_embeddings)

#retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 10})

/tmp/ipykernel_174160/3751257280.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(client=client, collection_name="ayudas",embedding_function=huggingface_embeddings)


In [4]:
collection = client.get_or_create_collection("ayudas")

query = """   Ayuntamiento de Madrid   """  
         # Sample question, change to other questions you are interested in.
# Ejemplo de búsqueda

results = collection.query(
    query_embeddings = np.array(huggingface_embeddings.embed_query(query)),
    #query_texts = [query],

    
    
    n_results=5,
    )


BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


In [5]:
pprint.pp(results['documents'])

[['Actualizado a: 30/12/2024Convocatoria de subvenciones para actividades '
  'comerciales, industriales, profesionales y para el autoempleo en el '
  'municipio de Ascó.1. Implementación, creación, rehabilitación y/o mejora, '
  'acondicionamiento y desarrollo de comercios, industrias y actividades '
  'profesionalesDestinatarios:Personas físicas o jurídicas titulares de '
  'comercios, industrias, de servicios y profesionalesInformación de '
  'interésÁmbito gegráfico:AscóOrganismo:Ayuntamiento de '
  'AscóAdministración:Administración LocalPlazo(s) de '
  'solicitud:31/12/2024Notas solicitud:Última convocatoria, hasta el 31 de '
  'diciembre de 2024. En espera de próxima '
  'convocatoriaTipo:SubvenciónImporte:Notas:Cuantía máxima de 30.000 '
  'eurosCEE:Referencias de la publicación- Acuerdo 240208. Butlletí Oficial de '
  'la Provincia de Tarragona número 33 de 15 de febrero de 2024. '
  '(Extracto-Convocatoria)Enlaces:SectoresSubsectoresDescriptoresComercioInfraestructuraMejora '

In [6]:
#llm=ChatGroq(groq_api_key=groq_api_key,
#model_name="gemma2-9b-it")
from langchain_community.llms import Ollama
llm = Ollama(model="gemma2:9b", base_url="http://127.0.0.1:11434")


/tmp/ipykernel_174160/3430781548.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma2:9b", base_url="http://127.0.0.1:11434")


In [7]:
""" #MULTIQUERY RETRIEVER

from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(search_kwargs={"k": 6}), llm=llm
) """

' #MULTIQUERY RETRIEVER\n\nfrom langchain.retrievers.multi_query import MultiQueryRetriever\nfrom langchain_openai import ChatOpenAI\n\nmultiquery_retriever = MultiQueryRetriever.from_llm(\n    retriever=db.as_retriever(search_kwargs={"k": 6}), llm=llm\n) '

In [8]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [9]:
retriever=db.as_retriever(search_kwargs={"k": 15})

In [10]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
compressor = CrossEncoderReranker(model=model, top_n=5)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke("¿Qué subvenciones da el Ayuntamiento de Madrid?")
pprint.pp(compressed_docs)

[Document(metadata={'Destinatarios': 'Titulares de concesión de los mercados municipales y asociaciones sin ánimo de lucro constituidas mayoritariamente por empresarios del sector comercial, hostelero y  hotelero', 'Organismo': 'Ayuntamiento de Madrid', 'Referencia': '100120 Título Convocatoria pública de subvenciones para fomentar la digitalización en el sector  comercial correspondiente a los años 2023, 2024 y 2025', 'Sector': 'Comercio, Empresas en general', 'Subsector': '', 'Tipo': 'Subvención', 'Ámbito Geográfico': 'Madrid'}, page_content='     13.7. Cuando fuera necesario establecer un orden de prelación de solicitudes, se incluirá una relación ordenada de todas aquellas que, cumpliendo con las condiciones administrativas y técnicas fijadas para adquirir la condición de beneficiario, no hayan sido estimadas por rebasarse la cuantía máxima del crédito fijado en la convocatoria, con indicación de la puntuación otorgad a a cada una de ellas en función de los criterios de valoración 

In [11]:
from langchain.chains import ReduceDocumentsChain

qa_template = """Eres un asistente para responder a preguntas "
    " basándote en los documentos proporcionados más abajo. "
    "Debes reproducir exactamente el fragmento de texto donde viene la respuesta"
    "Ordena todas las respuestas que encuentres en diferentes líneas,"
    "Si algún documento no contiene la respuesta, ignóralo."
    "\n\n"
    "{context}"

Pregunta: {question}
Respuesta:"""

prompt = PromptTemplate(template=qa_template,
                            input_variables=['context','question'])
combine_custom_prompt='''
Responde con todas las respuestas que encuentres en diferentes documentos.

Text:`{context}`
'''
""" 
""" 
combine_prompt_template = PromptTemplate(
    template=combine_custom_prompt, 
    input_variables=['context']
)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=compression_retriever, chain_type="map_reduce",return_source_documents=True,
 chain_type_kwargs= {
        "verbose": False,
        "question_prompt": prompt,
        "combine_prompt": combine_prompt_template,
        "combine_document_variable_name": "context"})

""" qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                          chain_type="stuff",
                                          retriever=compression_retriever,
                                          return_source_documents=True) """


query = """ ¿Qué subvenciones da el Ayuntamiento de Madrid?   """  
question = query

result=qa_chain.invoke(question)
pprint.pp(result)

ValueError: Ollama call failed with status code 500. Details: {"error":"llama runner process has terminated: signal: killed"}

In [ ]:
relevant_docs = result['source_documents']
print(f'There are {len(relevant_docs)} documents retrieved which are relevant to the query.')
print("*" * 100)
for i, doc in enumerate(relevant_docs):
    print(f"Relevant Document #{i+1}:\nSource file: {doc.metadata['source']}, Page: {doc.metadata['page']}\nContent: {doc.page_content}")
    print("-"*100)
    print(f'There are {len(relevant_docs)} documents retrieved which are relevant to the query.')

In [ ]:
print (relevant_docs)